In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue Oct 29 08:51:12 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [12]:
%env wd=/home/sam/analyses/20191029_pgen_v074.a4_repeats_counts
wd="/home/sam/analyses/20191029_pgen_v074.a4_repeats_counts"
gff_header = ['seqid','source','type','start','end','score','strand','phase','attributes']
%env rysnc_owl=owl:/volume1/web/halfshell/genomic-databank/
%env gff=Panopea-generosa-vv0.74.a4.repeat_region.gff3
%env wget_gffs=--directory-prefix=${wd} --recursive --quiety --no-directories --no-check-certificate --no-parent --accept 'Panopea-generosa-vv0.74.a4.repeat_region.gff3' https://owl.fish.washington.edu/halfshell/genomic-databank/
%env new_gff=Panopea-generosa-vv0.74.a4.repeats

# Set genome size to 942Mbp
GENOME_SIZE = 942000000


env: wd=/home/sam/analyses/20191029_pgen_v074.a4_repeats_counts
env: rysnc_owl=owl:/volume1/web/halfshell/genomic-databank/
env: gff=Panopea-generosa-vv0.74.a4.repeat_region.gff3
env: wget_gffs=--directory-prefix=$/home/sam/analyses/20191029_pgen_v074.a4_repeats_counts --recursive --quiety --no-directories --no-check-certificate --no-parent --accept 'Panopea-generosa-vv0.74.a4.repeat_region.gff3' https://owl.fish.washington.edu/halfshell/genomic-databank/
env: new_gff=Panopea-generosa-vv0.74.a4.repeats


In [13]:
# Calculate percentage of genome comprised of a given feature
def ind_repeats_percent(feature_length_sum): 
    return round(float(feature_length_sum / GENOME_SIZE * 100), 2)

### Import Python modules

In [14]:
import fnmatch
import os
import pandas

#### Create necessary directories

In [15]:
%%bash
mkdir --parents ${wd}

In [16]:
cd {wd}

/home/sam/analyses/20191029_pgen_v074.a4_repeats_counts


#### Download _Panopea generosa_ GFFs for v074.a3.

Info on GFFs is here: [https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome-feature-tracks-3)

In [17]:
%%bash

rsync \
--archive \
--verbose \
--progress \
--include="${gff}" \
--exclude="*" \
"${rysnc_owl}" \
.

echo ""
echo ""
echo "----------------------------------------------------------"

ls -lh

receiving incremental file list
./
Panopea-generosa-vv0.74.a4.repeat_region.gff3
    390,130,212 100%   28.74MB/s    0:00:12 (xfr#1, to-chk=0/2)

sent 91 bytes  received 390,177,991 bytes  25,172,779.48 bytes/sec
total size is 390,130,212  speedup is 1.00


----------------------------------------------------------
total 373M
-rwx------ 1 sam users 373M Oct 14 10:13 Panopea-generosa-vv0.74.a4.repeat_region.gff3


#### If need to download via wget, uncomment lines in the cell below

In [18]:
# %%bash
# time \
# wget "${wget_gffs}"

# ls -lh ${wd}

In [19]:
%%bash
head ${gff}

##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	1	225	1646	+	.	ID=19535.GS22252506.PGEN_.repeat00149391;Name=19535.GS22252506.PGEN_.repeat00149391;repeat_match=rnd-1_family-39;repeat_class=Unknown;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	910	1325	3459	+	.	ID=19535.GS22252506.PGEN_.repeat00149392;Name=19535.GS22252506.PGEN_.repeat00149392;repeat_match=rnd-1_family-135;repeat_class=Unknown;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	1329	2039	5278	-	.	ID=19535.GS22252506.PGEN_.repeat00149393;Name=19535.GS22252506.PGEN_.repeat00149393;repeat_match=rnd-6_family-1529;repeat_class=Unknown;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	3030	3608	3232	-	.	ID=19535.GS22252506.PGEN_.repeat00149394

In [20]:
%%bash

# Initialize array
features_array=()

# Identify unique features in GFF
## Store as an array
## Skip first three header lines, and then cut on two delimiters that are present
echo "Unique repeats features in ${gff}:"
while IFS='' read -r line 
do
    features_array+=("$line")
done < <(awk -F"class=" 'NR >3 {print $2}' ${gff} \
                  | sort -u \
                  | cut -d '%' -f 1 \
                  | cut -d ';' -f 1 \
                  | uniq)

# Check array contents
for feature in "${features_array[@]}"
do
    echo "${feature}"
done

# Loop through array and create new GFFs from each feature
for feature in "${features_array[@]}"
do
    echo "Parsing ${feature} from ${gff}..."
    echo "Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3"
    head -n 5 ${gff} > ${new_gff}.${feature}.gff3
    echo "Parsing matching feature lines for ${feature} feature..."
    grep "${feature}" ${gff} >> ${new_gff}.${feature}.gff3
    echo "Done with parsing ${feature} feature."
    feature_count=$(tail --lines +6 ${new_gff}.${feature}.gff3 | wc -l)
    echo "Identified ${feature_count} ${feature} features."
    echo "Output file is: Panopea-generosa-vv0.74.a3.repeats.${feature}.gff3"
    echo ""
done

echo "----------------------------------------------"
echo ""
ls -lh
  

Unique repeats features in Panopea-generosa-vv0.74.a4.repeat_region.gff3:
DNA
LINE
LTR
RC
Simple_repeat
SINE
Unknown
Parsing DNA from Panopea-generosa-vv0.74.a4.repeat_region.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.DNA.gff3
Parsing matching feature lines for DNA feature...
Done with parsing DNA feature.
Identified 21094 DNA features.
Output file is: Panopea-generosa-vv0.74.a3.repeats.DNA.gff3

Parsing LINE from Panopea-generosa-vv0.74.a4.repeat_region.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.LINE.gff3
Parsing matching feature lines for LINE feature...
Done with parsing LINE feature.
Identified 69365 LINE features.
Output file is: Panopea-generosa-vv0.74.a3.repeats.LINE.gff3

Parsing LTR from Panopea-generosa-vv0.74.a4.repeat_region.gff3...
Writing GFF3 header to Panopea-generosa-vv0.74.a3.repeats.LTR.gff3
Parsing matching feature lines for LTR feature...
Done with parsing LTR feature.
Identified 2890 LTR features.
Output file is: Panop

### Check the output files

In [21]:
%%bash
# Check the output files
for file in "${new_gff}"*.gff3
do
    echo ""
    echo ""
    echo "${file}"
    echo "----------------------------------------------"
    head "${file}"
done



Panopea-generosa-vv0.74.a4.repeats.DNA.gff3
----------------------------------------------
##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	1	225	1646	+	.	ID=19535.GS22252506.PGEN_.repeat00149391;Name=19535.GS22252506.PGEN_.repeat00149391;repeat_match=rnd-1_family-39;repeat_class=Unknown;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	910	1325	3459	+	.	ID=19535.GS22252506.PGEN_.repeat00149392;Name=19535.GS22252506.PGEN_.repeat00149392;repeat_match=rnd-1_family-135;repeat_class=Unknown;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_5d25089d78791-repeatmodeler	repeat_region	75698	76199	3502	-	.	ID=19535.GS22252506.PGEN_.repeat00149552;Name=19535.GS22252506.PGEN_.repeat00149552;repeat_match=rnd-1_family-397;repeat_class=DNA%2FhAT-hAT5;
PGA_scaffold2__36_contigs__length_69596280	GenSAS_

### Get sequence length stats for repeat features

In [22]:
total_repeats_percent = 0

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, 'Panopea-generosa-vv0.74.a4.repeats*.gff3'):
        print('\n' * 2)
        print(file)
        print("-------------------------")
        # Import GFF.
        # Skip first five rows and file is tab-separated
        gff=pandas.read_csv(file, header=None, skiprows=5, sep="\t")
        # Rename columns
        gff.columns = gff_header
        # Subtract start value from end value.
        # Have to add 1 so that sequence length can't equal zero
        gff['seqlength'] = gff.apply(lambda position: position['end'] - position['start'] + 1, axis=1)
        gff_sum = gff['seqlength'].sum()
        
        total_repeats_percent += ind_repeats_percent(gff_sum)
        print ("percent" , ind_repeats_percent(gff_sum))
        
        # Apply functions in list to seqlength column
        gff_stats = gff['seqlength'].agg(['sum', 'mean', 'min', 'median', 'max'])
        
        print (gff_stats.round(2))
print('\n' * 2) 
print("-------------------------")
print ("Repeats composition of genome (percent):" , total_repeats_percent)




Panopea-generosa-vv0.74.a4.repeats.LINE.gff3
-------------------------
percent 2.91
sum       27388849.00
mean           394.85
min             11.00
median         226.00
max           6604.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3
-------------------------
percent 0.5
sum       4733271.0
mean          261.2
min             6.0
median        125.0
max          5981.0
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3
-------------------------
percent 29.09
sum       2.740281e+08
mean      1.991900e+02
min       1.100000e+01
median    1.440000e+02
max       6.574000e+03
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a4.repeats.LTR.gff3
-------------------------
percent 0.22
sum       2060084.00
mean          712.83
min            11.00
median        316.00
max          6541.00
Name: seqlength, dtype: float64



Panopea-generosa-vv0.74.a4.repeats.RC.gff3
-------------------------
percent 0.0

In [23]:
%%bash
rm ${gff}
ls -lh

total 373M
-rw-rw-r-- 1 sam users 5.4M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.DNA.gff3
-rw-rw-r-- 1 sam users  18M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3
-rw-rw-r-- 1 sam users 735K Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3
-rw-rw-r-- 1 sam users 140K Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.RC.gff3
-rw-rw-r-- 1 sam users 4.5M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3
-rw-rw-r-- 1 sam users  11M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3
-rw-rw-r-- 1 sam users 335M Oct 29 08:52 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3
